##### Long term memory - basic usage

In [1]:
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3

conn = sqlite3.connect('scenarist.ai', check_same_thread=False)

store = InMemoryStore()
sqlite_store = SqliteSaver(conn=conn)

In [2]:
store.put(('users',), 'user01', {'username': 'kerem', 'job': 'rockstar'})

In [3]:
# sqlite_store.put(('documents',), 'doc01', {'name': 'sample doc'})

In [4]:
from langchain.tools import tool
from langgraph.types import interrupt, Command

@tool
def sample_func_that_requires_hitl():
    'This prints something crucial regarding to the story'
    
    said_you = interrupt("Approval needed! What say you?")
    
    print('this said_you: {}'.format(said_you))
    
    return 'well done'



In [5]:
from langchain.agents import create_agent

agent_w_memory = create_agent(
    'ollama:gpt-oss:20b', 
    checkpointer=sqlite_store,
    tools=[sample_func_that_requires_hitl],
    # interrupt_before=['tools'], # this interruption method is recommended only for debugging. it stops at the 'edges' of nodes
    system_prompt="You are an expert in screenwriting. You use what you can to give your best answer."
    )

In [6]:
config = {'configurable': {'thread_id': 1}}

In [8]:
response = agent_w_memory.invoke(
    {"messages": [{"role": "user", "content": "Print that crucial thing to the story"}]},
    config
    )

In [9]:
response

{'messages': [HumanMessage(content='What samples can you give me to start with my screenplay?', additional_kwargs={}, response_metadata={}, id='64a4e589-e15e-4972-8bd0-f378cba17ee7'),
  AIMessage(content='Below are a few ready‑to‑copy “starter samples” that you can drop straight into a screenwriter’s program (Final Draft, Celtx, WriterDuet, etc.) or paste into a plain‑text file.  \nThey’re intentionally **short** (one‑page scenes), **diverse** (genre‑wide), and **fully formatted** so you can see how a professional page looks. Feel free to tweak the wording, swap characters, or expand the beats.\n\n---\n\n## 1. Logline & Three‑Act Outline (so you know where the sample fits)\n\n| **Part** | **What Happens** | **Purpose** |\n|----------|------------------|-------------|\n| **Logline** | *When a disgraced forensic accountant, Maya, discovers a murder in a seemingly perfect small‑town bakery, she must navigate the town’s cozy façade to expose a conspiracy that reaches the city’s highest off

In [11]:
from IPython.display import display, Markdown

display(Markdown(response['messages'][-1].content))

In [13]:
snapshot = agent_w_memory.get_state(config)
next_action = snapshot.next
next_action

('tools',)

In [15]:
# continue the execution
from langgraph.types import Command

response_from_resume = agent_w_memory.invoke(Command(resume="I say yes darling. Lets do this!"), config)
response_from_resume

this said_you: I say yes darling. Lets do this!


{'messages': [HumanMessage(content='What samples can you give me to start with my screenplay?', additional_kwargs={}, response_metadata={}, id='64a4e589-e15e-4972-8bd0-f378cba17ee7'),
  AIMessage(content='Below are a few ready‑to‑copy “starter samples” that you can drop straight into a screenwriter’s program (Final Draft, Celtx, WriterDuet, etc.) or paste into a plain‑text file.  \nThey’re intentionally **short** (one‑page scenes), **diverse** (genre‑wide), and **fully formatted** so you can see how a professional page looks. Feel free to tweak the wording, swap characters, or expand the beats.\n\n---\n\n## 1. Logline & Three‑Act Outline (so you know where the sample fits)\n\n| **Part** | **What Happens** | **Purpose** |\n|----------|------------------|-------------|\n| **Logline** | *When a disgraced forensic accountant, Maya, discovers a murder in a seemingly perfect small‑town bakery, she must navigate the town’s cozy façade to expose a conspiracy that reaches the city’s highest off